In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import os

#device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #choose whether to use gpu or cpu

#hyper parameters
num_epochs = 30
batch_size = 1
learning_rate = 0.0005

#custom dataset
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = sorted(os.listdir(root_dir))
        self.labels = [self.extract_label(img) for img in self.images]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

    def extract_label(self, img_name):
        # Assuming that the label is part of the filename before the first underscore
        label = float(img_name[-17:-5])
        return label

# def get_data():
data_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder' #path to data

transform = transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  #transforms data to tensor and normalizes it

full_set = CustomImageDataset(root_dir=data_dir, transform=transform) #loads data from directory and applies transform
train_set, val_set, test_set = random_split(full_set, [int(len(full_set)*0.75), int(len(full_set)*0.15), int(len(full_set)*0.100056)]) #splits data into training, validation and test sets

train = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [6]:
dataiter = iter(train)
images, labels = next(dataiter)

conv1 = nn.Conv2d(3,6,5) #in_channels, out_channels, kernel_size
pool = nn.MaxPool2d(5,5) #kernel_size, stride (shift x pixel to the right)
conv2 = nn.Conv2d(6, 16, 5) 
# images = images.numpy()
print(images.shape)
x = conv1(images)
print(x.shape)
x = pool(x)
print(x.shape)
x = conv2(x)
print(x.shape)
x = pool(x)
print(x.shape)

torch.Size([1, 3, 100, 100])
torch.Size([1, 6, 96, 96])
torch.Size([1, 6, 19, 19])
torch.Size([1, 16, 15, 15])
torch.Size([1, 16, 3, 3])
